# Star with toruses

This is an example of a synthesized three dimensional volume
that is not easy to visualize using only two dimenaional projections.

The calculation to generate the volume array is not optimized and it takes a while to complete.

In [ ]:
import numpy as np
from numpy.linalg import norm

In [ ]:
def vec(*args):
    return np.array(args, dtype=np.float)

def normalize(V, epsilon=1e-12):
    nm = norm(V)
    if nm < epsilon:
        return vec(1, 0, 0) # whatever
    return (1.0 / nm) * V

def point_segment_distance(P, segment, epsilon=1e-4):
    A = segment[0]
    B = segment[1]
    V = B - A
    nV2 = V.dot(V)
    if (nV2 < epsilon):
        #print("degenerate segment")
        return norm(B - P)
    lmd = V.dot(P - A) / nV2
    if 0 < lmd and lmd < 1:
        #print("project onto line segment")
        Pprojection = A + lmd * V
        return norm(P - Pprojection)
    # otherwise distance to closer end point
    #print("closest end point")
    dA = norm(P - A)
    dB = norm(P - B)
    return min(dA, dB)

def circle_distance(P, center, radius, normal):
    plane_distance = normal.dot(P - center)
    vertical_offset = plane_distance * normal
    plane_projection = P - vertical_offset
    direction_from_center = normalize(plane_projection - center)
    circle_nearest_point = center + radius * direction_from_center
    distance = norm(P - circle_nearest_point)
    return distance

In [ ]:
circle_distance(vec(1,1,1), vec(0,1,1), 2, normalize(vec(-1,2,-1)))

In [ ]:
# tetrahedral vertices
vertices = vec([1,1,1], [-1,-1,1], [1,-1,-1], [-1,1,-1], )

normals = []
origin = vec(0,0,0)
radius = 0.9

for v in vertices:
    normals.append(normalize(v))
    
def inverse_distance_sum(P, epsilon=0.1):
    total = 0.0
    for (i, v) in enumerate(vertices):
        dsegment = point_segment_distance(P, [origin, v])
        total += 1.0 / (dsegment + epsilon)
        normal = normals[i]
        dcircle = circle_distance(P, v, radius, normal)
        total += 1.0 / (dcircle + epsilon)
    return total


for L in ([0,0,0], [-1, -1, -2], [1, -1, 1], [2,2,2], [1,1,1], [0,0,1]):
    print(L)
    P = vec(*L)
    print("   ", inverse_distance_sum(P))

In [ ]:
# This computation takes a while (it has not been optimized.)
N = 20
N2 = 2 * N
N4 = 4 * N
invN = 1.0 / N
A = np.zeros((N4, N4, N4), dtype=np.float)

def shifti(i):
    return (i - N2) * invN

for i in range(N4):
    for j in range(N4):
        for k in range(N4):
            P = shifti(vec(i,j,k))
            A[i, j, k] = inverse_distance_sum(P)

In [ ]:
from feedWebGL2 import volume
H = A.mean()
A.shape, H

In [ ]:
volume.widen_notebook()

W = volume.Volume32()

W

In [ ]:
W.load_3d_numpy_array(A, threshold=H, sorted=False)

#W.load_3d_numpy_array(A, threshold=H, sorted=True, method="diagonal")

x = W.build(1500)

In [ ]:
# Create a web folder containing the visualization
if False:
    target_folder = "../docs/torus_html"

    from feedWebGL2 import html_generator

    html_generator.generate_volume_html(A, target_folder, force=False, width=1500)